### 使用opensmile提取音频特征

In [ ]:
import opensmile
import pandas as pd
import os
import sklearn
import matplotlib.pyplot as plt
import numpy as np

## 1、构造特征提取函数

In [ ]:
## 提取特征函数
## file_list:音频文件路径的列表  list类型
## 返回值numpy.ndarray  形状:(len(file_list),88)
def extract_audio_feature(file_list):
    print("请耐心等待特征提取完！")
    smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.Functionals)
    feature = []
    for n,file in enumerate(file_list):
        y = smile.process_file(file)
        y = y.to_numpy().reshape(-1)
        feature.append(y)
        if (n+1)%100 == 0:
            print(f"当前进度{n+1}/{len(file_list)}")
    print("此次特征提取已结束")
    print("-------------------------------")
    feature = np.stack(feature,axis = 0)
    return feature

In [ ]:
## 特征提取示例
file = "./train/Ses01F_impro01_F000.wav"
audio_feature = extract_audio_feature([file])
print(type(audio_feature),audio_feature.shape)

## 2、csv文件读入示例

In [ ]:
## 读入csv文件示例
## 由于我们的csv文件使用"#"分隔，需要定义sep参数为"#",否则会读取失败！！！
train_csv = pd.read_csv("./CSVfile/train.csv",sep="#")

In [ ]:
## 快速查看前5条数据
train_csv.head(5)

In [ ]:
## 统计该csv下lable列不同值的数量
train_csv.value_counts(subset="label")

In [ ]:
## 画图显示类别数
plt.bar([0,1,2,3],list(train_csv.value_counts(subset="label")),tick_label = ["0","1","2","3"])
plt.xlabel("Label")
plt.ylabel("Num")
plt.title("Train dataset sample distribution")

## 3、性能指标函数

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

In [ ]:
def calculate_score_classification(preds, labels, average_f1='macro'):  # weighted, macro
    accuracy = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average=average_f1, zero_division=0)
    precision = precision_score(labels, preds, average='macro', zero_division=0)
    ua = recall_score(labels, preds, average='macro', zero_division=0)
    confuse_matrix = confusion_matrix(labels, preds)
    return accuracy, ua, f1, precision, confuse_matrix

### 4、构造机器学习模型

In [ ]:
##使用sklearn实现随机森林
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
## 这里的myrf类仅为示例，可以自行修改以达到更好的性能
class MyRF:
    def __init__(self):
        pass
    def train(self):
        pass
    def evaluate(self):
        pass
        

## 5、读取csv文件，分离音频文件路径、标签

In [ ]:
## 读取train.csv、dev.csv
train_csv = pd.read_csv("./CSVfile/train.csv", sep = "#")
dev_csv = pd.read_csv("./CSVfile/dev.csv", sep = "#")
## 分离文件路径和标签
## 可先截取少量样本验证代码正确性，再使用所有样本
# train_path = list(train_csv.path)[:100]
# train_label = list(train_csv.label)[:100]
# dev_path = list(dev_csv.path)[:50]
# dev_label = list(dev_csv.label)[:50]

train_path = list(train_csv.path)
train_label = list(train_csv.label)
dev_path = list(dev_csv.path)
dev_label = list(dev_csv.label)

## 6、结果写入函数

In [ ]:
## test_preds 长度为1241的list，对应测试集中1241个样本的标签
##运行后会在当前目录生成result.csv文件，提交result.csv文件即可
##如果没有生成，请检查test_preds的长度是否为1241！
def write_result(test_preds):
    if len(test_preds) != 1241:
        print("错误！请检查test_preds长度是否为1241！！！")
        return -1
    test_csv = pd.read_csv("./CSVfile/test.csv",sep="#")
    test_csv["label"] = test_preds
    test_csv.to_csv("./result.csv",sep = "#")
    print("测试集预测结果已成功写入到文件中！")

## 7、特征预处理函数、主函数

In [ ]:
## 特征处理函数，可以对提取的特征进行处理，以获得更好的特征表示
def feature_process(feature):
    return feature

## 主函数
if __name__ == "__main__":
    ## 实例化模型
    rf = MyRF()
    ## 提取训练样本特征
    ## 文件数量很多时需要的时间较长，请耐心等待
    train_feature = extract_audio_feature(train_path) ## np.array (n,88)
    train_feature = feature_process(train_feature)
    
    ##训练模型
    rf.train(train_feature,train_label)
    ##计算在dev上的性能
    dev_feature = extract_audio_feature(dev_path)
    dev_feature = feature_process(dev_feature)
    acc,ua,f1,pre,confuse_matrix = rf.evaluate(dev_feature,np.array(dev_label))
    print(f"Acc:{acc} \nUa:{ua} \nMacro_F1:{f1} \nPre:{pre}\nConfuse_matrix:\n{confuse_matrix}")
    
    ##读入test.csv
    test_csv = pd.read_csv("./CSVfile/test.csv",sep = "#")
    test_path = list(test_csv.path)
    test_feature = extract_audio_feature(test_path)
    test_label = rf.model.predict(test_feature)
    test_preds = test_label
    print(len(test_feature))
    
    ## 通过模型获得测试集对应的标签列表test_preds，并写入到result.csv文件中
    write_result(test_preds)